This file demonstrates how the Station data is created

In [14]:
pip install requests beautifulsoup4

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [21]:
pip install --upgrade requests

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/64.9 kB ? eta -:--:--
   ------ --------------------------------- 10.2/64.9 kB ? eta -:--:--
   ---------------------------------------- 64.9/64.9 kB 1.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [44]:
# Import necessary libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from datetime import datetime
import time

Step 1: Define URLs for London Tube lines

In [45]:
# These URLs are used to scrape station information for each tube line
base_url = "https://tfl.gov.uk/tube/route/"
elizabeth_url = "https://tfl.gov.uk/elizabeth-line/route/elizabeth"
lines = [
    {"name": "bakerloo", "url": f"{base_url}bakerloo/"},
    {"name": "central", "url": f"{base_url}central/"},
    {"name": "circle", "url": f"{base_url}circle/"},
    {"name": "district", "url": f"{base_url}district/"},
    {"name": "elizabeth-line", "url": elizabeth_url},  # Special case for Elizabeth Line
    {"name": "hammersmith-city", "url": f"{base_url}hammersmith-city/"},
    {"name": "jubilee", "url": f"{base_url}jubilee/"},
    {"name": "metropolitan", "url": f"{base_url}metropolitan/"},
    {"name": "northern", "url": f"{base_url}northern/"},
    {"name": "piccadilly", "url": f"{base_url}piccadilly/"},
    {"name": "victoria", "url": f"{base_url}victoria/"},
    {"name": "waterloo-city", "url": f"{base_url}waterloo-city/"},
]

Step 2: Scrape Tube station data

In [46]:
# Initialize an empty list to store station information
all_stations = []

In [47]:
# Iterate over each line to scrape data
for line in lines:
    try:
        # Send a request to the line's URL
        response = requests.get(line["url"])
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            print(f"Scraping line: {line['name']}")

            # Extract station data from the HTML
            stations = soup.select(".stop-list li")
            for station in stations:
                # Extract station name and StopNaptanId
                station_name = station.select_one(".stop-link").get_text(strip=True) if station.select_one(".stop-link") else None
                stop_naptan_id = station.select_one(".stopNaptanId")
                stop_naptan_id = stop_naptan_id["value"] if stop_naptan_id else None

                if station_name and stop_naptan_id:
                    # Clean the station name to remove unwanted content
                    station_name = re.sub(r"\s{3}.*", "", station_name).strip()
                    station_name = re.sub(r"Connect.*", "", station_name).strip()
                    
                    # Append the cleaned data to the list
                    all_stations.append({
                        "Line": line["name"].capitalize(),
                        "Station": station_name,
                        "StopNaptanId": stop_naptan_id
                    })
        else:
            print(f"Failed to fetch data for {line['name']} line, status code: {response.status_code}")
    except Exception as e:
        print(f"Error occurred while scraping {line['name']}: {e}")


Scraping line: bakerloo
Scraping line: central
Scraping line: circle
Scraping line: district
Scraping line: elizabeth-line
Scraping line: hammersmith-city
Scraping line: jubilee
Scraping line: metropolitan
Scraping line: northern
Scraping line: piccadilly
Scraping line: victoria
Scraping line: waterloo-city


Step 3: Save station data to Excel

In [48]:
# Convert the list of dictionaries into a DataFrame
stations_df = pd.DataFrame(all_stations)

In [64]:
# Display the DataFrame
stations_df.head()

,Line,Station,StopNaptanId
0,Bakerloo,Elephant & Castle Underground Station,940GZZLUEAC
1,Bakerloo,Lambeth North Underground Station,940GZZLULBN
2,Bakerloo,Waterloo Underground Station,940GZZLUWLO
3,Bakerloo,Embankment Underground Station,940GZZLUEMB
4,Bakerloo,Charing Cross Underground Station,940GZZLUCHX


In [49]:
# Create a timestamped filename for the output Excel file
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
stations_file = f"london_tube_stations_{timestamp}.xlsx"

In [50]:
# Save the DataFrame to an Excel file
stations_df.to_excel(stations_file, index=False)
print(f"Station data saved to '{stations_file}'")

Station data saved to 'london_tube_stations_20250128_144703.xlsx'


Step 4: Fetch latitude and longitude using TfL API

In [51]:
# Initialize a list to store API results
api_results = []
tfl_api_url = "https://api.tfl.gov.uk/StopPoint/"

In [52]:
# Read the station data from the previously saved Excel file
stations_df = pd.read_excel(stations_file)

In [53]:
# Iterate over each station to fetch location data
for index, row in stations_df.iterrows():
    stop_id = row['StopNaptanId']
    try:
        app_id = "I"
        app_key = "154d17a84edd4f5ba7c2350b556b3157"

        # Send a request to the TfL API
        response = requests.get(f"{tfl_api_url}{stop_id}?app_id={app_id}&app_key={app_key}")
        
        if response.status_code == 200:
            data = response.json()
            lat = data.get("lat", None)
            lon = data.get("lon", None)
            api_results.append({
                "StopNaptanId": stop_id,
                "Station": row['Station'],
                "Latitude": lat,
                "Longitude": lon
            })
        else:
            print(f"Failed to fetch data for StopNaptanId: {stop_id}, status code: {response.status_code}")
            api_results.append({
                "StopNaptanId": stop_id,
                "Station": row['Station'],
                "Latitude": None,
                "Longitude": None
            })

        # Add a delay to avoid hitting API rate limits
        time.sleep(0.5)

    except Exception as e:
        print(f"Error fetching data for StopNaptanId: {stop_id}: {e}")
        api_results.append({
            "StopNaptanId": stop_id,
            "Station": row['Station'],
            "Latitude": None,
            "Longitude": None
        })

Error fetching data for StopNaptanId: 940GZZLUMHL: HTTPSConnectionPool(host='api.tfl.gov.uk', port=443): Max retries exceeded with url: /StopPoint/940GZZLUMHL?app_id=I&app_key=154d17a84edd4f5ba7c2350b556b3157 (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))


Step 5: Save API results to DataFrame and display

In [54]:
# Convert the API results into a DataFrame
output_df = pd.DataFrame(api_results)

In [61]:
# Display the DataFrame
output_df.head()

,StopNaptanId,Station,Latitude,Longitude
0,940GZZLUEAC,Elephant & Castle Underground Station,51.494505,-0.099185
1,940GZZLULBN,Lambeth North Underground Station,51.498808,-0.112315
2,940GZZLUWLO,Waterloo Underground Station,51.504269,-0.113356
3,940GZZLUEMB,Embankment Underground Station,51.507058,-0.122666
4,940GZZLUCHX,Charing Cross Underground Station,51.507819,-0.126137


In [59]:
# Create a timestamped filename for the output Excel file
output_file = f"Stations_with_LatLon_{timestamp}.xlsx"

In [60]:
# Save the results to an Excel file
output_df.to_excel(output_file, index=False)
print(f"Latitude and Longitude data saved to '{output_file}'")

Latitude and Longitude data saved to 'Stations_with_LatLon_20250128_144703.xlsx'
